# II. Recommender Systems

## 1. Basics of Recommendation Algorithm

In [10]:
from scipy.spatial.distance import cosine
import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [34]:
M = np.asarray([[4,3,2,3], [1,2,3,1], [np.nan,2,1,np.nan], [4,3,2,np.nan]])

### Compute similarities

#### Cosine

In [35]:
import math
def cosine_similarity(v1,v2, metric='cosine'):
    if metric == 'correlation':
        v1 = v1 - np.nanmean(v1)
        v2 = v2 - np.nanmean(v2)
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        if np.isnan(x) or np.isnan(y): continue
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def sim_matrix(M, dimension='user', metric='cosine'):
    N = M.shape[0] if dimension == 'user' else M.shape[1]
    sim = np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            if i == j:
                sim[i,j] = 0
                continue
            if dimension == 'user':
                v1, v2 = M[i,:], M[j,:]
            else:
                v1, v2 = M[:,i], M[:,j]
            sim[i][j] = cosine_similarity(v1,v2,metric)
    return sim

In [36]:
cosine_similarity(M[0,:], M[2,:], 'cosine')

0.9922778767136677

In [37]:
sim_matrix(M, 'user')

array([[0.        , 0.79582243, 0.99227788, 1.        ],
       [0.79582243, 0.        , 0.86824314, 0.79406667],
       [0.99227788, 0.86824314, 0.        , 0.99227788],
       [1.        , 0.79406667, 0.99227788, 0.        ]])

In [38]:
sim_matrix(M, 'item')

array([[0.        , 0.9649505 , 0.80218063, 0.99705449],
       [0.9649505 , 0.        , 0.92450033, 0.96476382],
       [0.80218063, 0.92450033, 0.        , 0.78935222],
       [0.99705449, 0.96476382, 0.78935222, 0.        ]])

#### Pearson

In [39]:
cosine_similarity(M[0,:], M[2,:], 'correlation')

0.7071067811865475

In [40]:
sim_matrix(M, 'user', 'correlation')

array([[ 0.        , -0.85280287,  0.70710678,  1.        ],
       [-0.85280287,  0.        , -0.5547002 , -0.95618289],
       [ 0.70710678, -0.5547002 ,  0.        ,  0.70710678],
       [ 1.        , -0.95618289,  0.70710678,  0.        ]])

In [41]:
sim_matrix(M, 'item', 'correlation')

array([[ 0.        ,  0.94280904, -0.81649658,  0.9486833 ],
       [ 0.94280904,  0.        ,  0.        ,  1.        ],
       [-0.81649658,  0.        ,  0.        , -0.70710678],
       [ 0.9486833 ,  1.        , -0.70710678,  0.        ]])

### a) Compute the missing rating in this table using user-based collaborative filtering (CF). (Use cosine similarity, then use Pearson similarity). Assume taking all neighbors

In [42]:
def user_cf(M, metric='cosine'):
    pred = np.copy(M)
    n_users, n_items = M.shape
    avg_ratings = np.nanmean(M, axis=1)
    sim_users = sim_matrix(M, 'user', metric)
    for i in range(n_users):
        for j in range(n_items):
            if np.isnan(M[i,j]):
                #Get similarity values for current user
                cur_user_row = sim_users[i]
                highest = -99999
                highest_index = -1
                cur_index = 0
                #Iterate over values to find most similar index
                for x in cur_user_row:
                    if(highest < x or highest_index == -1):
                        highest = x;
                        highest_index = cur_index
                    cur_index += 1
                #Get row of most similar user and take their rating for item at column 'j'
                pred[i,j] = M[highest_index][j]
    return pred

In [43]:
print("User-based CF (Cosine): \n" + str(pd.DataFrame(user_cf(M, 'cosine'))))
print("User-based CF (Pearson): \n" + str(pd.DataFrame(user_cf(M, 'correlation'))))

User-based CF (Cosine): 
     0    1    2    3
0  4.0  3.0  2.0  3.0
1  1.0  2.0  3.0  1.0
2  4.0  2.0  1.0  3.0
3  4.0  3.0  2.0  3.0
User-based CF (Pearson): 
     0    1    2    3
0  4.0  3.0  2.0  3.0
1  1.0  2.0  3.0  1.0
2  4.0  2.0  1.0  3.0
3  4.0  3.0  2.0  3.0


### b) Similarly, computing the missing rating using item-based CF.

In [44]:
def item_cf(M, metric='cosine'):
    pred = np.copy(M)
    n_users, n_items = M.shape
    avg_ratings = np.nanmean(M, axis=0)
    sim_items = sim_matrix(M, 'item', metric)
    for i in range(n_users):
        for j in range(n_items):
            if np.isnan(M[i,j]):
                #Get similarity values for current item
                cur_item_row = sim_items[i]
                highest = -99999
                highest_index = -1
                cur_index = 0
                #Iterate over values to find most similar index
                for x in cur_item_row:
                    if(highest < x or highest_index == -1):
                        highest = x;
                        highest_index = cur_index
                    cur_index += 1
                #Get row of most similarly rated item and take their rating for user at column 'i'
                pred[i,j] = M[i][highest_index]
    return pred

In [45]:
print("Item-based CF (Cosine): \n" + str(pd.DataFrame(item_cf(M, 'cosine'))))
print("Item-based CF (Pearson): \n" + str(pd.DataFrame(item_cf(M, 'correlation'))))

Item-based CF (Cosine): 
     0    1    2    3
0  4.0  3.0  2.0  3.0
1  1.0  2.0  3.0  1.0
2  2.0  2.0  1.0  2.0
3  4.0  3.0  2.0  4.0
Item-based CF (Pearson): 
     0    1    2    3
0  4.0  3.0  2.0  3.0
1  1.0  2.0  3.0  1.0
2  2.0  2.0  1.0  2.0
3  4.0  3.0  2.0  3.0


## 2. Evaluating Recommendation Algorithms

### Predictive Accuracy

In [46]:
M_result = np.asarray([[4,3,2,3], 
                [1,2,3,1],
                [1,2,1,2],
                [4,3,2,4]])
pd.DataFrame(M_result)

,0,1,2,3
0,4,3,2,3
1,1,2,3,1
2,1,2,1,2
3,4,3,2,4


In [69]:
import math
def evaluateRS(ratings, groundtruth, method='user_cf', metric='cosine'):
    if method == 'user_cf':
        prediction = user_cf(ratings, metric)
    else:
        prediction = item_cf(ratings, metric)
    MSE = mean_squared_error(prediction, groundtruth)
    RMSE = round(math.sqrt(MSE),3)
    #print("RMSE using {0} approach ({2}) is: {1}".format(method, RMSE, metric))
    #print(pd.DataFrame(prediction))
    return MSE, RMSE

In [72]:
mse, rmse = evaluateRS(M, M_result, method='user_cf', metric='cosine')
print("MSE (Mean Square Error) using USER CF, Cosine metric: "+str(mse)+"; (Root: "+str(rmse)+")")

mse, rmse = evaluateRS(M, M_result, method='item_cf', metric='cosine')
print("MSE (Mean Square Error) using ITEM CF, Cosine metric: "+str(mse)+"; (Root: "+str(rmse)+")")

mse, rmse = evaluateRS(M, M_result, method='user_cf', metric='correlation')
print("MSE (Mean Square Error) using USER CF, Correlation metric: "+str(mse)+"; (Root: "+str(rmse)+")")

mse, rmse = evaluateRS(M, M_result, method='item_cf', metric='correlation')
print("MSE (Mean Square Error) using ITEM CF, Correlation metric: "+str(mse)+"; (Root: "+str(rmse)+")")

MSE (Mean Square Error) using USER CF, Cosine metric: 0.6875; (Root: 0.829)
MSE (Mean Square Error) using ITEM CF, Cosine metric: 0.0625; (Root: 0.25)
MSE (Mean Square Error) using USER CF, Correlation metric: 0.6875; (Root: 0.829)
MSE (Mean Square Error) using ITEM CF, Correlation metric: 0.125; (Root: 0.354)


### Evaluation
The lowest value given by this evaluation method is Item CF using the cosine metric. This indicates a better fit.

### Ranking Accuracy

In [73]:
import scipy.stats as stats

def evaluate_rank(ratings, groundtruth, method='user_cf', metric='cosine'):
    if method == 'user_cf':
        prediction = user_cf(ratings, metric)
    else:
        prediction = item_cf(ratings, metric)
    n_users = M.shape[0]
    avg_tau = 0
    for i in range(n_users):
        tau, p_value = stats.kendalltau(M_result[i,:], prediction[i,:])
        avg_tau += tau
    avg_tau = avg_tau / n_users
    #clear_output(wait=True)
    return avg_tau

print("Kendall's Tau Ranking Accuracy Score (USER CF, cosine) = " + str(evaluate_rank(M, M_result, 'user_cf', 'cosine')))
print("Kendall's Tau Ranking Accuracy Score (ITEM CF, cosine) = " + str(evaluate_rank(M, M_result, 'item_cf', 'cosine')))
print("Kendall's Tau Ranking Accuracy Score (USER CF, correlation) = " + str(evaluate_rank(M, M_result, 'user_cf', 'correlation')))
print("Kendall's Tau Ranking Accuracy Score (ITEM CF, correlation) = " + str(evaluate_rank(M, M_result, 'item_cf', 'correlation')))

#predicted_item_cf = item_cf(M, 'cosine')
#predicted_user_cf = user_cf(M, 'cosine')
#ground_truth = M_result

#concordant_i = 0
#discordant_i = 0

#concordant_u = 0
#discordant_u = 0

#for i in range(0,ground_truth.shape[0]):
#    s = 0
#    for j in range(0,ground_truth.shape[1]):
#        val_truth = ground_truth[i][j]
#        val_pred_i = predicted_item_cf[i][j]
#        val_pred_u = predicted_user_cf[i][j]
#        if()

Kendall's Tau Ranking Accuracy Score (USER CF, cosine) = 0.7
Kendall's Tau Ranking Accuracy Score (ITEM CF, cosine) = 0.8943375672974064
Kendall's Tau Ranking Accuracy Score (USER CF, correlation) = 0.7
Kendall's Tau Ranking Accuracy Score (ITEM CF, correlation) = 0.8443375672974064


### Conclusion
The accuracy ranking showed the best result using Item CF and the cosine metric. User CF remained unchanged between cosine and correlation metric types but most likely only due to lack of smaller decimal places.